# Chapter 6 Looping & Iteration

# 6.1 Writing Pythonic Loops
python 的迴圈和 c 語言寫法不同，這章節將講述如何寫出避免寫出 C-style loops  ，讓你的迴圈更 pythonic
  
以下範例就是經典的 C-style loops，用 i 當 index 及 len 來判斷該 iterate 幾次

In [1]:
my_items = ['a', 'b', 'c']
i = 0
while i < len(my_items):
  print(my_items[i])
  i += 1

a
b
c


pythonic 的寫法，將 i += 1 改成 range(len(_))

In [2]:
for i in range(len(my_items)):
  print(my_items[i])

# a better way to iter in python
for item in my_items:
  print(item)

a
b
c
a
b
c


如果真的需要 index

In [3]:
for i, item in enumerate(my_items):
  print(f"{i}: {item}")

0: a
1: b
2: c


用於 dictionary 的迴圈

In [4]:
emails = {
  'Bob': 'bob@example.com',
  'Alice': 'alice@example.com',
}

for name, email in emails.items():
  print(f"{name} - {email}")

Bob - bob@example.com
Alice - alice@example.com


需要間隔的迴圈使用以下寫法

```c
// c
for (i=0;i<10;i+=2){
  // do something
}
```

```python
# python
for i in range(0,10,2):
  pass
```

**Key Takeaways**
+ Writing C-style loops in Python is considered unpythonic.
Avoid managing **loop indexes** and stop conditions manually if
possible.
+ Python’s for-loops are really “for-each” loops that can iterate
directly over items from a container or sequence.

# 6.2 Comprehending Comprehensions
list comprehensions 譯作列表推導式，是用於創建 list / set 的方法  
   
其中一種方式為  
```python
values = [expression for item in collection]
```

In [5]:
squares = [x * x for x in range(10)]

# The above list comprehension is equal to following plain for-loop
squares = []
for x in range(10):
  squares.append(x * x)

在 list comprehensions 加入判斷條件

```python
values = [expression
  for item in collection
    if condition]
```

In [6]:
even_squares = [ x * x for x in range(10) if x % 2 == 0]

**Key Takeaways**
+ Comprehensions are a key feature in Python. Understanding
and applying them will make your code much more Pythonic.
+ Comprehensions are just fancy syntactic sugar for a simple forloop pattern. Once you understand the pattern, you’ll develop
an intuitive understanding for comprehensions.
+ There are more than just list comprehensions.

# 6.3 List Slicing Tricks and the Sushi Operator

可用 slicing 符號來存取元素

```python
# syntax
lst[start:end:step]
```

In [7]:
lst = [1, 2, 3, 4, 5]

# lst[start:end:step]
lst[1:3:1]

[2, 3]

反轉 list

In [8]:
lst[::-1]

[5, 4, 3, 2, 1]

清空 list 內的所有內容，但是保留 list，跟 list.clear (python3 才有)是一樣的事情

In [9]:
lst = [1,2,3,4,5]
del lst[:]

lst

[]

slice 同樣也能用於內容替換

In [10]:
original_lst = lst
lst[:] =[7, 8, 9]

print(lst)
print(original_lst)
print(original_lst is lst)

[7, 8, 9]
[7, 8, 9]
True


如果用等號的話仍指向相同object，slice 符號能建立 list 的 shallow copy

In [11]:
copied_lst = lst[:]

copied_lst is list

False

**Key Takeaways**
+ The : “sushi operator” is not only useful for selecting sublists
of elements within a list. It can also be used to clear, reverse,
and copy lists.
+ But be careful—this functionality borders on the arcane for
many Python developers. Using it might make your code less
maintainable for everyone else on your team

# 6.4 Beautiful Iterators
python iterator protocol:  只要實作 \_\_iter\_\_ and \_\_next\_\_ dunder 就能讓 class 在 for-loop 運作

## Iterating Forever
這是一個會無限印出 hello 的程式碼

In [12]:
class RepeaterIterator:
  def __init__(self, source):
    self.source = source
  def __next__(self):
    return self.source.value

In [13]:
class Repeater:
  def __init__(self, value):
    self.value = value
  
  def __iter__(self):
    return RepeaterIterator(self)

In [14]:
repeater = Repeater('Hello')

for i, item in enumerate(repeater): # 呼叫 repeater.__iter__() 取得 iterator
  # item = iterable_object.__next__()
  if i>10:
    break
  print(i, item)
  

0 Hello
1 Hello
2 Hello
3 Hello
4 Hello
5 Hello
6 Hello
7 Hello
8 Hello
9 Hello
10 Hello


程式碼背後的運作
+ 呼叫 `__iter__()` 來準備 iteration 所需要的 iterator object
+ 在每次的 loop，呼叫 iterator object 的 `__next__()` 來取得下一個數值

In [15]:
repeater = Repeater('Hello')
iterator = repeater.__iter__()

i=0
while True:
  item = iterator.__next__()
  print(item)
  
  if i == 10:
    break
  i += 1

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


你也可以模仿 loop 迴圈所做的事情來印出內容物

In [16]:
repeater = Repeater('Hello')
iterator = iter(repeater)

next(iterator)

'Hello'

## A Simpler Iterator Class
我們的範例中包含了兩個物件 Reperter 和 RepeaterIterator，他們對應到 python 的兩個 iterator 機制。
+ 透過 `iter()` 取得 iterator object
+ 在迴圈重複呼叫 `next()` 取值

在多數情況其實可以寫成同一個 class 來減少程式碼

In [17]:
class Repeater:
  def __init__(self, value):
    self.value = value
  
  def __iter__(self):
    return self
  
  def __next__(self):
    return self.value

## Who Wants to Iterate Forever
無限迴圈的 iterator 並不實用，我們希望設計出能離開迴圈的 iterator，  
以下的 my_list 在印出內容後會 raise stop StopIteration ，這是 python 用於告知跳離迴圈的方式


In [18]:
my_list = [1, 2, 3]

iterator = iter(my_list)

next(iterator)
next(iterator)
next(iterator)

# StopIteration 
# next(iterator)

3

於是有了以下的實作，執行達到 max_repeats 次數後 raise StopIteration 

In [19]:
class BoundedRepeater:
  def __init__(self, value, max_repeats):
    self.value = value
    self.max_repeats = max_repeats
    self.count = 0

  def __iter__(self):
    return self
    
  def __next__(self):
    if self.count >= self.max_repeats:
      raise StopIteration
    self.count += 1
    return self.value

In [20]:
repeater = BoundedRepeater('hello', 3)

for item in repeater:
  print(item)

hello
hello
hello


**Key Takeaways**
+ Iterators provide a sequence interface to Python objects that’s
memory efficient and considered Pythonic. Behold the beauty
of the for-in loop!
+ To support iteration an object needs to implement the iterator protocol by providing the \_\_iter\_\_ and \_\_next\_\_ dunder
methods.
+ Class-based iterators are only one way to write iterable objects
in Python. Also consider generators and generator expressions.

# 6.5 Generators Are Simplified Iterators
如果你認為寫一堆的程式碼，就是為了重複輸出一句話有點大材小用，那麼 **generator** 將會是你的好夥伴


In [21]:
def repeater(value):
  while True:
    yield value

In [22]:
i=0
for x in repeater('Hi'):
  print(x)
  if i == 10:
    break
  i += 1

Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi


呼叫 repeater function ，實際上是建立並回傳了一個 generator object

In [23]:
repeater('key')

<generator object repeater at 0x7f52ab14d4d0>

yield 的作用是暫停 function ，保留 local state 並回傳數值  
隨時都能呼叫 `next` 來取得下一個狀態，這也使得 generator 符合 iterator protocal

In [24]:
iterator = repeater('Hi')
next(iterator)

'Hi'

## Generators That Stop Generating
讓我們來看一下把 while True 去掉會發生甚麼事情  
+ 在每次的 yield 都會暫停狀態並回傳 local state，接著往下繼續執行

In [25]:
def repeat_three_times(value):
  yield value
  yield value
  yield value

In [26]:
for x in repeat_three_times('hello'):
  print(x)

hello
hello
hello


從範例得知，只要 function 結束就會終止迴圈  


In [27]:
iterator = repeat_three_times('hello')
next(iterator)
next(iterator)
next(iterator)

# StopIteration
# next(iterator)

'hello'

我們能進一步把程式碼改進，超過次數就結束 function 停止迴圈

In [28]:
def bounded_repeater(value, max_repeats):
  for i in range(max_repeats):
    yield value

**Key Takeaways**
+ Generator functions are **syntactic sugar** for writing objects that
support the iterator protocol. Generators abstract away much
of the boilerplate code needed when writing class-based iterators.
+ The yield statement allows you to temporarily suspend execution of a generator function and to pass back values from it.
+ Generators start raising StopIteration exceptions after control flow leaves the generator function by any means other than
a yield statement.


# 6.6 Generator Expressions
Generator expressions give you an even more **effective shortcut** for
writing iterators. With a simple and concise syntax that looks like a
list comprehension, you’ll be able to define iterators in a **single line** of
code  


以下的一行程式碼能直接取代第二個範例，感受到 generator expressions 的強大了嗎

In [29]:
iterator = ('hello' for i in range(3))

In [30]:
def bounded_repeater(value, max_repeats):
  for i in range(max_repeats):
    yield value
iterator = bounded_repeater('Hello', 3)

## Generator Expressions vs List Comprehensions
generator object 跟 list 的創建手法非常相近，但他們是不同的東西  

```python
genexpr = (expression for item in collection)
```
上面的 template 對應到 generator function
```python
def generator():
  for item in collection:
    yield expression
```

In [31]:
listcomp = ['Hello' for i in range(3)]
genexpr = ('Hello' for i in range(3))

In [32]:
listcomp

['Hello', 'Hello', 'Hello']

In [33]:
iterator

<generator object bounded_repeater at 0x7f52ab14d750>

generator object 能轉換成 list

In [34]:
list(iterator)

['Hello', 'Hello', 'Hello']

## Filtering Values
如同建立 list 一樣，你也能加入一些條件

```python
genexpr = (expression for item in collection
      if condition)
```
對應
```
def generator():
  for item in collection:
    if condition:
      yield expression
```

In [35]:
even_squares = (x * x for x in range(10)
          if x % 2 == 0)

## In-line Generator Expressions
由於 generator 是 expression，因此能用 in-line 方式使用

In [36]:
for x in ('Bom dia' for i in range(3)):
  print(x)  

Bom dia
Bom dia
Bom dia


能讓程式碼更簡潔的作法: 將 generator expression 的括號去除

In [37]:
sum((x * 2 for x in range(10)))

# Versus:

sum(x * 2 for x in range(10))

90

## Too Much of a Good Thing…
generator expressions 也允許巢狀迴圈，但這不利於維護，建議最多兩層就好

```python
(expr for x in xs if cond1
  for y in ys if cond2
  ...
  for z in zs if condN)
```
對應到的邏輯
```
for x in xs:
  if cond1:
    for y in ys:
      if cond2:
      ...
        for z in zs:
          if condN:
            yield expr
```

**Key Takeaways**
+ Generator expressions are **similar to list comprehensions.**
However, they don’t construct list objects. Instead, generator
expressions generate values “just in time” like a class-based
iterator or generator function would.
+ Once a generator expression has been consumed, **it can’t be
restarted or reused.**
+ Generator expressions are best for implementing simple **“ad
hoc” iterators.** For complex iterators, it’s better to write a generator function or a class-based iterator

# 6.7 Iterator Chains
Here’s another great feature of iterators in Python:  
By **chaining together multiple iterators** you can write highly efficient data processing “pipelines.

In [38]:
def integers():
  for i in range(1, 9):
    yield i

In [39]:
def squared(seq):
  for i in seq:
    yield i * i

In [40]:
def negated(seq):
  for i in seq:
    yield -i

In [41]:
chain = squared(integers())
list(chain)

[1, 4, 9, 16, 25, 36, 49, 64]

In [42]:
list(negated(squared(integers())))

[-1, -4, -9, -16, -25, -36, -49, -64]

In [43]:
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)

negated

<generator object <genexpr> at 0x7f52ab14d350>

In [44]:
list(negated)

[0, -1, -4, -9, -16, -25, -36, -49]

**Key Takeaways**
+ Generators can be chained together to form highly efficient and
maintainable data processing pipelines.
+ Chained generators process each element going through the
chain individually.
+ Generator expressions can be used to write concise pipeline definitions, but this can impact readability.